In [1]:
import os
os.chdir("/home/oo7433/FIND/oo7433/NPDiscovery/MicroMGx/")

PermissionError: [Errno 13] Permission denied: '/home/oo7433/FIND/oo7433/NPDiscovery/MicroMGx/'

In [ ]:
with open('metabologenomics.txt', 'r') as f:
    strains = f.read().strip().split('\n')

In [ ]:
import pandas as pd
gcfs = pd.read_table("gcfs_data.txt", sep = "\t", index_col = 0)
dict_gcf = dict(zip(gcfs.index.to_list(), gcfs['bgcs'].str.split().tolist()))
import re
dict_bgc = {}
for gcf in dict_gcf.keys():
    bgcs = list(filter(lambda x: any([re.match(strain, x) for strain in strains]), dict_gcf[gcf]))
    if len(bgcs) > 0:
        for bgc in bgcs:
            if bgc in dict_bgc:
                dict_bgc[bgc].append(gcf)
            else:
                dict_bgc[bgc] = [gcf]

In [ ]:
import json
with open('gcfs.json', 'w') as f:
    json.dump(dict_gcf, f)

In [2]:
antismash_dir = "/home/oo7433/FIND/MicroMGx_Pilot/genomics/antismash_outputs/"
output_dir = "bgc_table/"

In [ ]:
from bs4 import BeautifulSoup
from pathlib import Path
from glob import glob
for strain in strains:
    with open(antismash_dir + strain + "/index.html", 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    df = pd.DataFrame(columns = ['Region', 'Type', 'From', 'To', 'Contig Edge Warning', 'GenBank', 'ID', 'GCFs'])
    count = 0
    overview = soup.find('div', {'class': 'page', 'id': 'overview'})
    for contig in overview.find_all('div', {'class': 'record-overview-details'}):
        for region in contig.find('tbody').find_all('tr'):
            info = region.find_all('td')
            name = info[0].attrs['class'][-1]
            Region = info[0].a.text.replace('&nbsp', ' ')
            Type = info[0].attrs['class'][1].replace(',', ', ')
            From = info[2].text
            To = info[3].text
            details = soup.find('div', {'class': 'page', 'id': name})
            GenBank = details.find('div', {'class': 'region-download'}).a.attrs['href']
            Warning = details.find('div', {'class': 'contig-edge-warning'})
            Warning = Warning.text.strip('.') if Warning else ''
            if not Path(antismash_dir + strain + '/' + GenBank).is_file():
                print(f'{strain}: GenBank file {GenBank} cannot be found in the directory!')
            count += 1
            ID = strain + '_' + str(count)
            GCFs = ', '.join(dict_bgc.get(ID, ''))
            df.loc[name] = [Region, Type, From, To, Warning, GenBank, ID, GCFs]
    gbkfiles = glob(antismash_dir + strain + '/*.gbk')
    if count != (len(gbkfiles) - 1):
        print(strain, ':', str(len(gbkfiles) - 1), "GenBank files in the directory,", count, "GenBank files in the table.")
    df.to_csv(output_dir + strain + "_bgc_table.csv", index = False)

In [ ]:
from ipywidgets import interact
import pandas as pd
@interact(strain = strains)
def bgc_table(strain = 'DA936Z0011'):
    df = pd.read_csv(output_dir + strain + "_bgc_table.csv", keep_default_na = False)
    df = df.drop(['GenBank'], axis = 1)
    df.index = range(1, df.shape[0] + 1)
    styles = [
        dict(selector = 'th', props = [('font-size', '110%'), ('text-align', 'center')]),
        dict(selector = 'td', props = [('text-align', 'center')])
    ]
    return df.style\
        .apply(lambda x: ['color: red' if v else '' for v in x], subset = ['Contig Edge Warning'])\
        .set_properties(**{'font-size': '8pt'})\
        .set_table_styles(styles)